<a href="https://colab.research.google.com/github/hosngholami/RecommenderSystem/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors

pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 100)

In [ ]:
hotel = pd.read_csv('/content/drive/MyDrive/Shiraz/hotel.csv')
historic =  pd.read_csv('/content/drive/MyDrive/Shiraz/historic.csv')
museum = pd.read_csv('/content/drive/MyDrive/Shiraz/museum.csv')
park = pd.read_csv('/content/drive/MyDrive/Shiraz/park.csv')
restaurant = pd.read_csv('/content/drive/MyDrive/Shiraz/restaurant.csv')
hospital = pd.read_csv('/content/drive/MyDrive/Shiraz/hospital.csv')
placeOfWorship = pd.read_csv('/content/drive/MyDrive/Shiraz/place_of_worship.csv')
swimmingPool = pd.read_csv('/content/drive/MyDrive/Shiraz/swimming_pool.csv')
caffe = pd.read_csv('/content/drive/MyDrive/Shiraz/caffe.csv')

In [ ]:
hotel["type"] = "hotel"
historic["type"] = "historic"
museum["type"] = "museum"
park["type"] = "park"
restaurant["type"] = "restaurant"
hospital["type"] = "hospital"
placeOfWorship["type"] = "placeofworship"
swimmingPool["type"] = "swimmingpool"
caffe["type"] = "caffe"

In [ ]:
hotel = hotel[["name", "name:en", "stars", "tourism", "geometry"]]
hotel.to_csv('/content/drive/MyDrive/Shiraz/hotel.csv')
historic.to_csv('/content/drive/MyDrive/Shiraz/historic.csv')
museum.to_csv('/content/drive/MyDrive/Shiraz/museum.csv')
park.to_csv('/content/drive/MyDrive/Shiraz/park.csv')
restaurant.to_csv('/content/drive/MyDrive/Shiraz/restaurant.csv')
hospital.to_csv('/content/drive/MyDrive/Shiraz/hospital.csv')
placeOfWorship.to_csv('/content/drive/MyDrive/Shiraz/placeOfWorship.csv')
swimmingPool.to_csv('/content/drive/MyDrive/Shiraz/swimmingPool.csv')
caffe.to_csv('/content/drive/MyDrive/Shiraz/caffe.csv')


In [ ]:
historic = historic[["amenity",
          "building",
          "heritage",
          "historic",
          "leisure",
          "museum",
          "memorial",
          "name",
          "name:en",
          "opening_hours",
          "tourism",
          "geometry"]]
historic.to_csv('/content/drive/MyDrive/Shiraz/historic.csv')

In [ ]:
museum = museum[["amenity", "building", "heritage", "historic", "museum", "tourism", "geometry"]]
museum.to_csv('/content/drive/MyDrive/Shiraz/museum.csv')

In [ ]:
park = park[["leisure", "name", "name:en", "opening_hours", "geometry"]]
park.to_csv('/content/drive/MyDrive/Shiraz/park.csv')

In [ ]:
restaurant = restaurant[["amenity", "branch", "building", "cuisine", "name", "name:en", "opening_hours", "tourism", "geometry"]]
restaurant.to_csv('/content/drive/MyDrive/Shiraz/restaurant.csv')

In [ ]:
hospital = hospital[["amenity",
          "name",
          "name:en",
          "opening_hours",
          "geometry"]]

hospital.to_csv('/content/drive/MyDrive/Shiraz/hospital.csv')

In [ ]:
placeOfWorship = placeOfWorship[["amenity",
                "historic",
                "name",
                "name:en",
                "opening_hours",
                "place_of_worship:type",
                "tourism",
                "geometry"]]

placeOfWorship.to_csv('/content/drive/MyDrive/Shiraz/placeOfWorship.csv')

In [ ]:
locations = pd.concat([historic,
                       museum,
                       park,
                       restaurant,
                       placeOfWorship,
                       swimmingPool,
                       caffe])
locations["Latitude"] = 0
locations["Longitude"] = 0

In [ ]:
locations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1223 entries, 0 to 344
Columns: 104 entries, Unnamed: 0.1 to Longitude
dtypes: float64(5), int64(2), object(97)
memory usage: 1003.2+ KB


In [ ]:
for i in range(len(locations)):
  location = locations.iloc[i]['geometry']
  latitude = str(location).split(" ")[1]
  latitude = latitude.replace("'", "")
  latitude = latitude.replace("(", "")

  longitude = location.split(" ")[2]
  longitude = longitude.replace("'", "")
  longitude = longitude.replace(")", "")
  longitude = longitude.replace("(", "")


  locations.loc[i, 'Latitude'] = latitude
  locations.loc[i, 'Longitude'] = longitude






In [ ]:
locations.to_csv('/content/drive/MyDrive/Shiraz/locations.csv')

In [4]:
locations = pd.read_csv('/content/drive/MyDrive/Shiraz/locations.csv')

In [ ]:
tag = pd.DataFrame(columns=["Id", "title:en", "title:fn"])

In [ ]:
data = [
{
    "Id": 1,
    "Title": "Historical",
    "Title:Fn" : "مکان‌های تاریخی مانند آثار باستانی و معماری قدیمی."
},{
    "Id": 2,
    "Title": "Recreational",
    "Title:Fn" : "مکان‌های تفریحی مانند پارک‌ها، شهربازی‌ها و مراکز خرید."
},{
    "Id": 3,
    "Title": "Natural",
    "Title:Fn" : "جاذبه‌های طبیعی مانند کوه‌ها، جنگل‌ها، دریاها و آبشارها."
},{
    "Id": 4,
    "Title": "Adventure",
    "Title:Fn" : "مکان‌های ماجراجویانه (مانند کوه‌نوردی، صخره‌نوردی)."
},{
    "Id": 5,
    "Title": "Religious",
    "Title:Fn" : "مکان‌های مذهبی (مانند کلیساها، مساجد، معابد)."
},{
    "Id": 6,
    "Title": "Health/Thermal",
    "Title:Fn" : "مکان‌های سلامت و درمانی مانند چشمه‌های آب گرم"
}, {
    "Id": 7,
    "Title": "Resort",
    "Title:Fn" : "استراحتگاه."
},
{
    "Id": 8,
    "Title": "Artistic",
    "Title:Fn" : "مکان‌هایی هنری مانند گالری‌ها، موزه‌ها و مناظر زیبا."
},
{
    "Id": 9,
    "Title": "Photography",
    "Title:Fn" : " مکان های مناسب عکاسی مانند گالری، موزه ها و مناظر زیبا."
},
{
    "Id": 10,
    "Title": "Cultural",
    "Title:Fn" : "مکان‌هایی که فرهنگ و هنر یک منطقه را نمایش می‌دهند."
},
]

In [ ]:
tag = pd.read_csv('/content/drive/MyDrive/Shiraz/tag.csv')

In [ ]:
tag

,Unnamed: 0,Id,Title,Title:Fn
0,0,1,Historical,مکان‌های تاریخی مانند آثار باستانی و معماری قد...
1,1,2,Recreational,مکان‌های تفریحی مانند پارک‌ها، شهربازی‌ها و مر...
2,2,3,Natural,جاذبه‌های طبیعی مانند کوه‌ها، جنگل‌ها، دریاها ...
3,3,4,Adventure,مکان‌های ماجراجویانه (مانند کوه‌نوردی، صخره‌نو...
4,4,5,Religious,مکان‌های مذهبی (مانند کلیساها، مساجد، معابد).
5,5,6,Health/Thermal,مکان‌های سلامت و درمانی مانند چشمه‌های آب گرم
6,6,7,Resort,استراحتگاه.
7,7,8,Artistic,مکان‌هایی هنری مانند گالری‌ها، موزه‌ها و مناظر...
8,8,9,Photography,مکان های مناسب عکاسی مانند گالری، موزه ها و م...
9,9,10,Cultural,مکان‌هایی که فرهنگ و هنر یک منطقه را نمایش می‌...


In [38]:
# locations[locations["type"] == "historic"]['tag'] = "historic"
locations["tag"] = [[] for _ in range(len(locations))]

print(set(locations["amenity"]))
for i in range(len(locations)):
  if (locations.iloc[i]["amenity"] == 'cafe'):
    if('caffe' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('caffe')

  if (locations.iloc[i]["amenity"] == 'restaurant'):
    if('restaurant' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('restaurant')

  if (locations.iloc[i]["amenity"] == 'theatre'):
    if('Artistic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Artistic')
    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')


  if (locations.iloc[i]["amenity"] == 'place_of_worship'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')




{'cafe', 'marketplace', 'place_of_worship', 'theatre', 'public_bath', nan, 'restaurant'}


In [39]:
filtered_df = locations[locations['tag'].apply(lambda x: 'Cultural' in x)]
filtered_df["tag"]

,tag
92,"[Artistic, Cultural]"


In [40]:
set(locations["building"])

for i in range(len(locations)):
  if (locations.iloc[i]["building"] == 'mosque'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')

  if (locations.iloc[i]["building"] == 'mausoleum'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')
    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["building"] == 'sports_hall'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

In [43]:
print(type(locations["heritage"]))

for i in range(len(locations)):
  if (locations.iloc[i]["heritage"] == 2):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')

  if (locations.iloc[i]["building"] == 'mausoleum'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')
    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["building"] == 'sports_hall'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

<class 'pandas.core.series.Series'>


In [48]:
print(set(locations["historic"]))
for i in range(len(locations)):
  if (locations.iloc[i]["historic"] == 'caravanserai'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["historic"] == 'castle'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'castle_wall'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'city_gate'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["historic"] == 'garden'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'manor'):

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'monument'):

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'pillory'):

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'yes'):

    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')




{'building', 'tomb', 'manor', 'yes', 'pillory', 'تپه_پوستچی', 'city_gate', 'garden', 'monument', 'house', 'caravanserai', 'castle_wall', nan, 'castle', 'memorial'}


In [51]:
set(locations["leisure"])
for i in range(len(locations)):
  if (locations.iloc[i]["leisure"] == 'garden'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')
  if (locations.iloc[i]["leisure"] == 'park'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

  if (locations.iloc[i]["leisure"] == 'swimming_pool'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')


In [53]:
set(locations["museum"])

{'art', 'history', 'music', 'music;ceramics', nan}